# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [128]:
def read_col_file(filename):
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [129]:
import random
from itertools import permutations
from copy import copy

In [130]:
def count_edges(edges, cut):
    ans = 0
    for edge in edges:
        if not(set(edge) <= cut[0] or set(edge) <= cut[1]):
            ans += 1
    return ans

In [131]:
def get_random_cut(vert):
    part = set(random.sample(vert, len(vert) // 2))
    cut = [part, vert - part]
    return cut

In [132]:
def find_nearest(edges, cut, k):
    test = copy(cut)
    ans = copy(cut)
    best_dist = count_edges(edges, cut)
    for first in permutations(cut[0], k):
        for second in permutations(cut[1], k):
            pool_set = set(first) | set(second)
            for i in permutations(pool_set, k):
                left = set(i)
                right = copy(pool_set) - left
                test[0].union(left).difference(right)
                test[1].union(right).difference(left)
                dist = count_edges(edges, test)
                if dist < best_dist:
                    best_dist, ans = dist, copy(test)
    return ans

In [133]:
def basic_local_search(graph):
    vertices = graph[0]
    edges = graph[1]
    cut = get_random_cut(vertices)
    current = count_edges(edges, cut)
    while True:
        cut = find_nearest(edges, cut, 1)
        dist = count_edges(edges, cut)
        if dist >= current:
            break
        else:
            current = dist
    return cut[0]

In [134]:
print(basic_local_search(read_col_file("myciel3.col")))

{1, 3, 6, 8, 10}


In [135]:
print(basic_local_search(read_col_file("myciel4.col")))

{1, 2, 4, 5, 6, 10, 11, 13, 15, 17, 23}


In [136]:
print(basic_local_search(read_col_file("myciel5.col")))

{1, 2, 3, 4, 7, 10, 11, 13, 18, 19, 20, 21, 26, 28, 30, 31, 33, 34, 35, 39, 40, 41, 45}


In [137]:
print(basic_local_search(read_col_file("myciel6.col")))

{1, 5, 7, 8, 10, 12, 16, 19, 21, 22, 23, 24, 25, 30, 31, 32, 35, 36, 37, 39, 41, 42, 45, 49, 50, 51, 52, 53, 55, 58, 60, 61, 62, 63, 67, 68, 74, 76, 80, 81, 85, 86, 87, 89, 93, 94, 95}


In [138]:
print(basic_local_search(read_col_file("myciel7.col")))

{1, 2, 3, 7, 8, 11, 12, 13, 15, 18, 23, 24, 25, 26, 27, 30, 31, 34, 35, 37, 40, 41, 42, 43, 44, 46, 47, 48, 50, 53, 54, 56, 58, 60, 61, 63, 64, 67, 69, 71, 77, 78, 79, 80, 82, 83, 88, 89, 91, 92, 93, 94, 97, 98, 99, 105, 107, 108, 109, 110, 111, 112, 116, 121, 122, 124, 125, 129, 130, 131, 132, 133, 134, 138, 144, 145, 147, 148, 150, 152, 153, 155, 157, 158, 161, 162, 170, 174, 178, 179, 182, 185, 186, 188, 190}
